In [1]:
import tensorflow as tf
from tensorflow import keras

In [2]:
import pandas as pd
import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras import layers
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn import tree
import xgboost as xb

#Hyperparameter tuning modules
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RandomizedSearchCV
from scipy.stats import randint as sp_randint
from tensorflow.keras.wrappers.scikit_learn import KerasRegressor
from sklearn.metrics import mean_squared_error, mean_absolute_error
from sklearn.metrics import make_scorer
import matplotlib.pyplot as plt
%matplotlib inline

from tensorflow.keras.callbacks import EarlyStopping

C:\Users\William\Anaconda3\lib\site-packages\dask\dataframe\utils.py:14: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [3]:
import warnings
warnings.filterwarnings("ignore")

In [4]:
# Reading training datasets
frequency = pd.read_csv('frequency_domain_features_train.csv')
heart_rate = pd.read_csv('heart_rate_non_linear_features_train.csv')
time_domain = pd.read_csv('time_domain_features_train.csv')

In [5]:
train = heart_rate.merge(frequency, how = 'inner', on = 'uuid')
train = train.merge(time_domain, how = 'inner', on = 'uuid')

In [116]:
train['HR'].head()

0    69.499952
1    64.363150
2    67.450066
3    68.809562
4    74.565728
Name: HR, dtype: float64

In [7]:
train.drop(['uuid', 'condition'], inplace=True, axis=1)

In [8]:
train.head()

,SD1,SD2,sampen,higuci,datasetId,VLF,VLF_PCT,LF,LF_PCT,LF_NU,...,KURT,SKEW,MEAN_REL_RR,MEDIAN_REL_RR,SDRR_REL_RR,RMSSD_REL_RR,SDSD_REL_RR,SDRR_RMSSD_REL_RR,KURT_REL_RR,SKEW_REL_RR
0,11.001565,199.061782,2.139754,1.163485,2,2661.894136,72.203287,1009.249419,27.375666,98.485263,...,-0.856554,0.335218,-0.000203,-0.000179,0.017080,0.007969,0.007969,2.143342,-0.856554,0.335218
1,9.170129,114.634458,2.174499,1.084711,2,2314.265450,76.975728,690.113275,22.954139,99.695397,...,-0.408190,-0.155286,-0.000059,0.000611,0.013978,0.004769,0.004769,2.930855,-0.408190,-0.155286
2,11.533417,118.939253,2.135350,1.176315,2,1373.887112,51.152225,1298.222619,48.335104,98.950472,...,0.351789,-0.656813,-0.000011,-0.000263,0.018539,0.008716,0.008716,2.127053,0.351789,-0.656813
3,11.119476,127.318597,2.178341,1.179688,2,2410.357408,70.180308,1005.981659,29.290305,98.224706,...,-0.504947,-0.386138,0.000112,0.000494,0.017761,0.008660,0.008660,2.050988,-0.504947,-0.386138
4,13.590641,87.718281,2.221121,1.249612,2,1151.177330,43.918366,1421.782051,54.242160,96.720007,...,-0.548408,-0.154252,-0.000100,-0.002736,0.023715,0.013055,0.013055,1.816544,-0.548408,-0.154252


In [9]:
train.shape

(369289, 35)

In [10]:
# Dropping columns Joe suggested
cols = ['datasetId', 'SD2', 'SDRR_RMSSD_REL_RR', 'SDRR_REL_RR', 'MEAN_REL_RR', 'RMSSD_REL_RR', 'MEDIAN_REL_RR', 'RMSSD', 'LF_PCT', 'SKEW_REL_RR', 'HF', 'LF_NU', 'HF_NU', 'SD1', 'KURT_REL_RR']
train.drop(cols, inplace=True, axis=1)

In [11]:
# Train/valid/test split
X = train.drop('HR', axis=1)
y = train['HR']

In [93]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=7)

In [13]:
# Creating function to design model

def design_model(X=X_train, learning_rate=5):
    model = Sequential(name='model')
    input = tf.keras.Input(shape=(X.shape[1],))
    model.add(input)
    # Adding one hidden layer
    model.add(layers.Dense(128, activation='relu'))
    model.add(layers.Dropout(0.1))
    model.add(layers.Dense(64, activation='relu'))
    model.add(layers.Dropout(0.2))
    model.add(layers.Dense(32, activation='relu'))
    model.add(layers.Dropout(0.3))
    model.add(layers.Dense(1))
    opt = tf.keras.optimizers.Adam(learning_rate=learning_rate)
    model.compile(loss='mse', metrics=['mae'], optimizer=opt)
    return model

In [14]:
y_train = np.asarray(y_train)
y_test = np.asarray(y_test)

In [15]:
def fit_model(model, X_train, y_train, num_epochs):
    es = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=20)
    history = model.fit(X_train, y_train, epochs=num_epochs, batch_size=30, verbose=1, validation_split=0.3, callbacks=[es])
    return history

In [16]:
model = design_model(X_train, learning_rate=0.5)
model.summary()

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 128)               2560      
_________________________________________________________________
dropout (Dropout)            (None, 128)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 64)                8256      
_________________________________________________________________
dropout_1 (Dropout)          (None, 64)                0         
_________________________________________________________________
dense_2 (Dense)              (None, 32)                2080      
_________________________________________________________________
dropout_2 (Dropout)          (None, 32)                0         
_________________________________________________________________
dense_3 (Dense)              (None, 1)                 33    

In [17]:
#fit_model(model, X_train, y_train, num_epochs=30)

In [18]:
#model.fit(X_train, y_train, validation_split=0.3, verbose=1)

In [19]:
val_mse, val_mae = model.evaluate(X_test, y_test, verbose=0)

In [20]:
print("MAE: ", val_mae)

MAE:  280.3956604003906


In [21]:
model = design_model(X_train, learning_rate=2)

In [22]:
model

In [23]:
# Grid Search
def do_grid_search():
    batch_size = [6, 64]
    epochs = [10, 40]
    model = KerasRegressor(build_fn=design_model)
    param_grid = dict(batch_size=batch_size, epochs=epochs)
    grid = GridSearchCV(estimator=model, verbose=1, n_jobs=-1, param_grid=param_grid, scoring = make_scorer(mean_absolute_error, greater_is_better=False), return_train_score=True)
    grid_result = grid.fit(X_train, y_train, verbose=0)
    grid_result.best_estimator_
    print(grid_result)
    print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))
    
    means = grid_result.cv_results_['mean_test_score']
    stds = grid_result.cv_results_['std_test_score']
    params = grid_result.cv_results_['params']
    
    for mean, stdev, param in zip(means, stds, params):
        print("%f (%f) with: %r" % (mean, stdev, param))
        
    print("Training")
    
    means = grid_result.cv_results_['mean_train_score']
    stds = grid_result.cv_results_['std_train_score']
    params = grid_result.cv_results_['params']
    
    for mean, stdev, param in zip(means, stds, params):
        print("%f (%f) with: %r" % (mean, stdev, param))

In [25]:
#do_grid_search()

In [ ]:
#-----------------Automated Stacking----------------------

In [26]:
from vecstack import stacking

In [78]:
models = [
    LinearRegression(n_jobs=-1),
    #RandomForestRegressor(random_state=7, max_depth=3),
    xb.XGBRegressor(random_state=7, learning_rate=0.5, n_estimators=100, max_depth=3)
]

In [79]:
s_train, s_test = stacking(models, X_train, y_train, X_test, regression=True, random_state=7, verbose=1, n_folds=5, stratified=True)

task:         [regression]
metric:       [mean_absolute_error]
mode:         [oof_pred_bag]
n_models:     [2]

model  0:     [LinearRegression]
    ----
    MEAN:     [0.97043693] + [0.00559822]
    FULL:     [0.97043693]

model  1:     [XGBRegressor]
    ----
    MEAN:     [0.23966975] + [0.00680685]
    FULL:     [0.23966973]



In [80]:
stacked_model = xb.XGBRegressor(random_state=7)

stacked_model = model.fit(s_train, y_train)

y_pred_stacked = stacked_model.predict(s_test)

print("Final MAE score: [%.8f]" % mean_absolute_error(y_test, y_pred_stacked))

Final MAE score: [0.16968266]


In [82]:
stacked_mae = mean_absolute_error(y_test, y_pred_stacked)

In [113]:
y_preds = pd.DataFrame(data=y_pred_stacked, columns=['Stacking Pred'], index = X_test.index.copy())
y_preds = pd.merge(train['HR'], y_preds, how='right', left_index=True, right_index=True)

In [118]:
y_preds.reset_index(drop=True)

,HR,Stacking Pred
0,73.980817,73.917488
1,64.062723,64.167236
2,67.593559,67.563293
3,79.994453,79.941299
4,54.921336,55.469967
...,...,...
110782,86.813935,86.651764
110783,78.122775,77.721497
110784,66.340979,66.192177
110785,68.910415,68.789719


In [ ]:
# Finally figured it out - HR column is actual values